## 使用Whoosh实现简单的文本检索
文本检索是很多NLP任务的重要组成部分，也是信息检索（IR, Information Retrieval）领域的重要研究问题。本文并不考虑复杂的NLP任务，也不考虑IR研究中涉及的复杂问题，仅从“简单够用”的角度，给出一种实现参考。

这里用到了开源软件[Whoosh](http://pythonhosted.org/Whoosh/intro.html "Whoosh")，这是一个Python实现的搜索服务框架，此处使用的是Whoosh 2.6.0版本。

测试用到的语料是[PFR人民日报语料库](http://www.icl.pku.edu.cn/icl_groups/corpus/dwldform1.asp "PFR人民日报语料库")，通过预处理我只保留了分词结果。在后面的测试中，我认为每一个完整的句子（句号分隔）是一条记录。

（不要在意这些细节，这只是个简单的示例^(*￣(oo)￣)^）

### ① 设定Whoosh的Schema
这个Schema指定了记录的标题、内容、标识符等信息的类型，由于我们的目标是文本检索，所以标题和内容都是TEXT，Schema还有很多别的设定，比如可以设定记录的关键字等等。
变量ix是搜索的关键：索引。

In [1]:
import os

from whoosh.index import create_in
from whoosh.fields import *

schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT)
ix = create_in("Exp0131_files" + os.path.sep + "index", schema)

### ② 读入语料，建立索引
这是实现搜索的一个重要手段，即对每一条记录建立索引记录，为了示意方便，我们把每一条记录的内容都写到title中，内容content留空。

In [2]:
idx = 1
writer = ix.writer()
with open("Exp0131_files" + os.path.sep + "textbase.txt") as fin:
    for line in fin:
        writer.add_document(title=line.decode('utf-8'), path=u"Id: %d" % idx,content=u'N/A')
        idx += 1
        
writer.commit()
print '%d records loaded.' % idx

231189 records loaded.


### ③ 检索测试
索引建立完成之后，就可以进行检索测试，默认的Whoosh支持简单的AND OR用法，这里简单示意一个如下

In [5]:
%%time
from whoosh.qparser import QueryParser

with ix.searcher() as searcher:
    query = QueryParser("title", ix.schema).parse(u"新闻 AND 美国 OR 日本")
    results = searcher.search(query)
    if results:
        for res in results:
            print res['title']
    else:
        print 'Not found'

有 一个 现象 引起 了 我 的 注意 ，今年 来 采访 “ 两会 ” 的 外国 记者 中 ，前 7 位 的 国家 是 日本 、 美国 、 英国 、 德国 、 韩国 、 法国 、 加拿大 ，而 日本 、 美国 的 主要 新闻 媒体 派出 的 记者 近 200 人 ，占 采访 “ 两会 ” 外国 记者 总数 的 一半 ，说明 发达国家 对 中国 的 关注 程度 。

当 记者 今晨 在 饭桌 上 与 美国 《 新闻 周刊 》 和 法国 《 费加罗 报 》 记者 交换 看法 时 ，新闻 已 成为 “ 旧闻 ” 。

中国 和 美国 远 隔 重洋 ，两 国 人民 间 的 相互 了解 很多 要 通过 新闻 媒体 。

我 知道 美国 人民 不 会 这么 做 的 ，怎么 能 把 战争 物资 卖 给 一个 侵略 国家 呢 ？‘ 是的 ，’ 他 肯定 地 说 ，‘ 消息 来源 ，就 是 你们 美国 的 新闻 电讯 。

美国 《 新闻 周刊 》 甚至 用 “ 德国 病 ” 这样 的 概念 描述 德国 经济 现状 。

他 在 会谈 后 举行 的 新闻 发布会 上 表示 ，美国 将 继续 支持 埃及 的 军事 现代化 计划 。

奥尔布赖特 在 抵达 科威特 后 接受 美国 有线 新闻 广播 公司 记者 采访 时 说 ，美国 并 不 认为 ，美国 需要 联合国 安理会 通过 一 项 新 的 决议 之后 才 能 对 伊拉克 动武 。

例如 ，日本 的 索尼 公司 在 80 年代 末 以后 大举 投资 新闻 媒体 和 娱乐业 ，在 1989年 买 下 美国 哥伦比亚 影片 和 法国 的 特莱玛 音乐 制作 公司 ，随后 同 美国 的 布洛克巴斯特 影视 公司 和 时代 — 沃纳 公司 合资 建立 了 几 个 广播 公司 ，还 与 一些 电视 、 电影 制作 公司 进行 了 合资 。

我们 国家 也 有 犯罪 ，也 有 流浪 街头 的 孩子 ，却 没 人 过问 。 ”美国 新闻 代表团 团长 曾 这样 感慨 。

最新 一 期 《 美国 新闻 和 世界 报道 》 周刊 援引 美国 众议院 的 报告 说 ，伊拉克 曾 于 1991年 夏 把 化学武器 和 400 枚 飞毛腿 导弹 转移 到 也门 和 苏丹 。

Wall time: 179 ms


## 结语
很显然，本文仅仅是一个最简单的入门示例，这个过程中存在很多问题，例如：
- 检索的过程中，并没有加入检索词的模糊性，目前的检索要求检索词必须精确匹配
- 索引和检索的速度很慢，在上面的例子中，建立索引大概用了五分钟，单次检索的用时也在百毫秒级别（只有二十多万记录而已）

这些问题在实际中大概需要逐一解决，值得庆幸的是Whoose在设计时就把自己定位在一个框架，因此功能的扩展（估计）并不会很难。最后引用Whoose文档中的一段话，用来说明Whoose的定位：

> Whoosh is a fast, pure Python search engine library.

>The primary design impetus of Whoosh is that it is pure Python. You should be able to use Whoosh anywhere you can use Python, no compiler or Java required.

>Like one of its ancestors, Lucene, Whoosh is not really a search engine, it’s a programmer library for creating a search engine [1].

>Practically no important behavior of Whoosh is hard-coded. Indexing of text, the level of information stored for each term in each field, parsing of search queries, the types of queries allowed, scoring algorithms, etc. are all customizable, replaceable, and extensible.

>[1]	It would of course be possible to build a turnkey search engine on top of Whoosh, like Nutch and Solr use Lucene.